In [6]:
import numpy as np
from IPython.display import Audio
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2Model, Wav2Vec2Config, Wav2Vec2Tokenizer, Wav2Vec2FeatureExtractor
import torchaudio
from transformers import Wav2Vec2Model

# change the directory to the root of the parent folder
import os
os.chdir("..")

In [11]:
data = np.load('speech_waveform/stimuli/audiobook_1.npz')
waveform = data['audio']
waveform = waveform[0:int(len(waveform)* 0.05)]

# Check and convert the waveform to a PyTorch tensor if it's not already
if not isinstance(waveform, torch.Tensor):
    waveform = torch.tensor(waveform)

print(waveform.shape)

torch.Size([2248390])


In [10]:
# Load the pre-trained Wav2Vec 2.0 model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h")

# Resample waveform if needed
if waveform.shape[-1] != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
    waveform = resampler(waveform)
    
# Load your audio file (replace 'path_to_your_audio_file' with your audio file path)
audio_input = waveform
sample_rate = 16000

# Process the audio input
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# Get the latent representation
with torch.no_grad():
    latent_representation = model(input_values).last_hidden_state

# Print the shape of the output
print(latent_representation.shape)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 233, 1024])


In [29]:
# 1. Load the model and the tokenizer
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53")
# Load the feature extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")

print(feature_extractor)
# Resample waveform if needed
if waveform.shape[-1] != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
    waveform = resampler(waveform)

print(waveform.squeeze().numpy().shape)
# Process the waveform
input_values = feature_extractor(waveform, return_tensors="pt", padding="longest", sampling_rate=16000, do_normalize=True).input_values

# Get the latent representation
with torch.no_grad():
    embeddings = model(input_values)

print(embeddings.keys())
print(embeddings.last_hidden_state.shape)
print(embeddings.extract_features.shape)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

(749464,)
odict_keys(['last_hidden_state', 'extract_features'])
torch.Size([1, 2341, 1024])
torch.Size([1, 2341, 512])
